In [1]:
!nvidia-smi

Tue Oct  1 17:43:31 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.09                 Driver Version: 561.09         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   38C    P4             13W /   30W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\HP\Desktop\fall


## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **18.01.2023** with version **YOLOv8.0.9**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [3]:
# Pip install method (recommended)

!pip3 install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.1  Python-3.12.6 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Setup complete  (16 CPUs, 31.6 GB RAM, 142.2/952.8 GB disk)


In [4]:
from ultralytics import YOLO

from IPython.display import display, Image

## CLI Basics 

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://v8docs.ultralytics.com/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

## Inference with Pre-trained COCO Model

### 💻 CLI

`yolo mode=predict` runs YOLOv8 inference on a variety of sources, downloading models automatically from the latest YOLOv8 release, and saving results to `runs/predict`.

In [6]:
%cd {HOME}
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' device='cuda'

c:\Users\HP\Desktop\fall
Ultralytics 8.3.1 🚀 Python-3.12.6 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

Found https://media.roboflow.com/notebooks/examples/dog.jpeg locally at dog.jpeg
image 1/1 c:\Users\HP\Desktop\fall\dog.jpeg: 640x384 1 person, 1 car, 1 dog, 58.8ms
Speed: 3.4ms preprocess, 58.8ms inference, 178.0ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs\detect\predict3
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
# %cd {HOME}
# Image(filename='runs/detect/predict/dog.jpeg', height=600)

### 🐍 Python SDK

The simplest way of simply using YOLOv8 directly in a Python environment.

In [7]:
model = YOLO(f'{HOME}/yolov8n.pt')
results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25, device='cuda')


Found https://media.roboflow.com/notebooks/examples/dog.jpeg locally at dog.jpeg
image 1/1 c:\Users\HP\Desktop\fall\dog.jpeg: 640x384 1 person, 1 car, 1 dog, 72.3ms
Speed: 0.0ms preprocess, 72.3ms inference, 108.1ms postprocess per image at shape (1, 3, 640, 384)


In [8]:
results[0].boxes.xyxy

tensor([[   0.0000,  314.5476,  625.0081, 1278.2410],
        [  55.2331,  250.0977,  648.0330, 1266.2471],
        [ 633.2305,  719.5388,  701.0454,  786.0400]], device='cuda:0')

In [9]:
results[0].boxes.conf

tensor([0.7269, 0.2902, 0.2840], device='cuda:0')

In [10]:
results[0].boxes.cls

tensor([ 0., 16.,  2.], device='cuda:0')

## Custom Training

In [11]:
!pip3 install ultralytics torch torchvision

In [17]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


 DATASET

In [1]:
!pip3 install roboflow

In [1]:

from roboflow import Roboflow
rf = Roboflow(api_key="I6Dg08bETrwTKkOyF8U9")
project = rf.workspace("yolov5-lw9t6").project("broken-trees")
version = project.version(1)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ft2-2 in yolov8:: 100%|██████████| 16334/16334 [00:07<00:00, 2322.60it/s]


In [28]:
!pip3 install datasets

In [1]:
from datasets import load_dataset

ds = load_dataset("dbutt7/NTP_Treefall_Segmentation")

c:\Users\HP\Desktop\fall\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [5]:
import os

label_dir = 'C:\\Users\\HP\\Desktop\\fall\\broken-trees-1\\train\\labels'  # Update this with your labels directory

for label_file in os.listdir(label_dir):
    if label_file.endswith('.txt'):
        with open(os.path.join(label_dir, label_file)) as f:
            for line in f:
                parts = line.split()
                if parts:  # Check if line is not empty
                    try:
                        # Attempt to convert the first part to int
                        class_id = int(parts[0])
                    except ValueError:
                        print(f"Invalid class id in {label_file}: '{parts[0]}' in line: '{line.strip()}'")



In [2]:
import os
import torch
from ultralytics import YOLO

# Change to your desired working directory
os.chdir('C:\\Users\\HP\\Desktop\\fall')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = YOLO('yolov8s.pt')

# Run YOLO training on CPU
model.train(
    data = 'C:\\Users\\HP\\Desktop\\fall\\data.yaml',
    epochs = 50,
    imgsz = 640,
    device = device,
    batch = 4,
    workers = 4
)

model.save('yolov8_custom.pt')


New https://pypi.org/project/ultralytics/8.3.9 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.3  Python-3.12.6 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\HP\Desktop\fall\data.yaml, epochs=50, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=4, project=None, name=train37, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=N

train: Scanning C:\Users\HP\Desktop\fall\broken-trees-1\train\labels.cache... 988 images, 0 backgrounds, 0 corrupt: 100%|██████████| 988/988 [00:00<?, ?it/s]
val: Scanning C:\Users\HP\Desktop\fall\broken-trees-1\valid\labels.cache... 283 images, 0 backgrounds, 0 corrupt: 100%|██████████| 283/283 [00:00<?, ?it/s]


Plotting labels to runs\detect\train37\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 63 weight(decay=0.0), 70 weight(decay=0.0005), 69 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs\detect\train37
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      1.22G      2.001      3.412      2.093          7        640: 100%|██████████| 247/247 [00:20<00:00, 11.83it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:02<00:00, 15.28it/s]

                   all        283        355     0.0827      0.159     0.0577      0.017


...

      50/50      1.24G      1.668      1.774      1.937          4        640: 100%|██████████| 247/247 [00:19<00:00, 12.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:02<00:00, 16.28it/s]

                   all        283        355      0.488      0.523      0.473      0.192



50 epochs completed in 0.353 hours.
Optimizer stripped from runs\detect\train37\weights\last.pt, 19.9MB
Optimizer stripped from runs\detect\train37\weights\best.pt, 19.9MB

Validating runs\detect\train37\weights\best.pt...
Ultralytics 8.3.3  Python-3.12.6 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Model summary (fused): 186 layers, 9,828,825 parameters, 0 gradients, 23.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:01<00:00, 18.20it/s]


                   all        283        355      0.484      0.501      0.461      0.194
                     0        177        203      0.491      0.614      0.541      0.232
                     1        139        152      0.477      0.388       0.38      0.155
Speed: 0.2ms preprocess, 3.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\train37


In [4]:
import os

# Path to the directory
path = 'runs/detect/train/'

# List files in the directory
files = os.listdir(path)
print(files)


['args.yaml', 'labels.jpg', 'labels_correlogram.jpg', 'train_batch0.jpg', 'train_batch1.jpg', 'train_batch2.jpg', 'weights']


In [13]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load and display the image
img = mpimg.imread('C:\\Users\\HP\\Desktop\\fall\\runs\\detect\\train9\\confusion_matrix.png')
plt.figure(figsize=(10, 6))  # Optional: specify figure size
plt.imshow(img)

plt.show()


<Figure size 1000x600 with 1 Axes>

In [14]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load and display the image
img = mpimg.imread('C:\\Users\\HP\\Desktop\\fall\\runs\\detect\\train9\\results.png')
plt.figure(figsize=(10, 6))  # Optional: specify figure size
plt.imshow(img)

plt.show()


<Figure size 1000x600 with 1 Axes>

In [15]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load and display the image
img = mpimg.imread('C:\\Users\\HP\\Desktop\\fall\\runs\\detect\\train9\\val_batch0_pred.jpg')
plt.figure(figsize=(10, 6))  # Optional: specify figure size
plt.imshow(img)

plt.show()


<Figure size 1000x600 with 1 Axes>

## Validate Custom Model

In [16]:
import torch
from ultralytics import YOLO

# Check if CUDA is available
if torch.cuda.is_available():
    device = '0'  # Use first GPU
else:
    device = 'cpu'  # Fall back to CPU
    print("CUDA is not available. Using CPU.")

# Load the YOLO model with the best weights
model = YOLO('runs/detect/train9/weights/best.pt')

# Validate the model using the specified dataset
results = model.val(data='data.yaml', device=device)

# Print results
print(results)


Ultralytics 8.3.1  Python-3.12.6 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Model summary (fused): 186 layers, 9,828,825 parameters, 0 gradients, 23.3 GFLOPs


val: Scanning C:\Users\HP\Desktop\fall\broken-trees-1\valid\labels.cache... 283 images, 0 backgrounds, 0 corrupt: 100%|██████████| 283/283 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  4.66it/s]


                   all        283        355      0.513      0.481      0.454      0.185
                     0        177        203      0.533       0.64      0.539      0.228
                     1        139        152      0.492      0.322      0.369      0.141
Speed: 0.6ms preprocess, 8.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C6126C8140>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,   

## Inference with Custom Model

In [14]:
!pip3 install supervision

   ---------------------------------------- 0.0/151.5 kB ? eta -:--:--
   ---------- ---------------------------- 41.0/151.5 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 151.5/151.5 kB 3.0 MB/s eta 0:00:00


In [16]:
!yolo task=detect mode=predict model=runs/detect/train9/weights/best.pt conf=0.25 source='C:\\Users\\HP\\Desktop\\fall\\Fall_Testing\\tree_collaps\\t1.mp4'

Ultralytics 8.3.3 🚀 Python-3.12.6 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Model summary (fused): 186 layers, 9,828,825 parameters, 0 gradients, 23.3 GFLOPs

video 1/1 (frame 1/330) C:\Users\HP\Desktop\fall\Fall_Testing\tree_collaps\t1.mp4: 384x640 3 0s, 34.8ms
video 1/1 (frame 2/330) C:\Users\HP\Desktop\fall\Fall_Testing\tree_collaps\t1.mp4: 384x640 3 0s, 4.8ms
video 1/1 (frame 3/330) C:\Users\HP\Desktop\fall\Fall_Testing\tree_collaps\t1.mp4: 384x640 3 0s, 10.1ms
video 1/1 (frame 4/330) C:\Users\HP\Desktop\fall\Fall_Testing\tree_collaps\t1.mp4: 384x640 3 0s, 10.1ms
video 1/1 (frame 5/330) C:\Users\HP\Desktop\fall\Fall_Testing\tree_collaps\t1.mp4: 384x640 3 0s, 5.6ms
video 1/1 (frame 6/330) C:\Users\HP\Desktop\fall\Fall_Testing\tree_collaps\t1.mp4: 384x640 3 0s, 5.5ms
video 1/1 (frame 7/330) C:\Users\HP\Desktop\fall\Fall_Testing\tree_collaps\t1.mp4: 384x640 3 0s, 0.0ms
video 1/1 (frame 8/330) C:\Users\HP\Desktop\fall\Fall_Testing\tree_collaps\t1.mp4: 384x6

**NOTE:** Let's take a look at few results.

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob('runs/detect/predict/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

In [12]:
!pip3 uninstall opencv-python

^C


In [19]:
!pip3 install --upgrade ultralytics


In [6]:
!pip3 install --upgrade opencv-python opencv-contrib-python


   ---------------------------------------- 0.0/45.5 MB ? eta -:--:--
   - -------------------------------------- 1.3/45.5 MB 11.3 MB/s eta 0:00:04
   ---- ----------------------------------- 4.7/45.5 MB 15.9 MB/s eta 0:00:03
   ----------- ---------------------------- 13.1/45.5 MB 24.9 MB/s eta 0:00:02
   --------------- ------------------------ 17.3/45.5 MB 24.2 MB/s eta 0:00:02
   ------------------ --------------------- 21.5/45.5 MB 22.7 MB/s eta 0:00:02
   ---------------------- ----------------- 25.4/45.5 MB 22.1 MB/s eta 0:00:01
   ------------------------- -------------- 29.1/45.5 MB 21.7 MB/s eta 0:00:01
   ----------------------------- ---------- 33.6/45.5 MB 21.5 MB/s eta 0:00:01
   --------------------------------- ------ 37.7/45.5 MB 21.4 MB/s eta 0:00:01
   ------------------------------------ --- 41.7/45.5 MB 21.2 MB/s eta 0:00:01
   ---------------------------------------  45.4/45.5 MB 21.2 MB/s eta 0:00:01
   ---------------------------------------- 45.5/45.5 MB 20.3 M

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\HP\\Desktop\\fall\\.venv\\Lib\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [15]:
import os

model_path = 'C:\\Users\\HP\\Desktop\\fall\\runs\\detect\\train9\\weights\\best.pt'
print(os.path.isfile(model_path))  # Should return True if the file exists


True


In [16]:
from ultralytics import YOLO

# Load your custom YOLOv8 model with weights
model = YOLO('C:\\Users\\HP\\Desktop\\fall\\runs\\detect\\train9\\weights\\best.pt')


c:\Users\HP\Desktop\fall\.venv\Lib\site-packages\ultralytics\nn\tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Perform a forward pass through the model.


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="I6Dg08bETrwTKkOyF8U9")
project = rf.workspace("ukasha-rahman-y7ad7").project("disaster-detection")
version = project.version(17)
dataset = version.download("yolov8")


In [22]:
!pip3 install cvzone

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26311 sha256=54809d0bb53b1d5a0ef363f8aada13def0bc500e9f6423412d316e9d84a80fdc
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\5d\21\e8\3147ae88d44e27f06e0175d337a7673c70fb957202cbbe2034
Successfully built cvzone


FINAL STEP WITH BOUNDING BOX?

In [10]:
import cv2
import os
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('C:\\Users\\HP\\Desktop\\fall\\runs\\detect\\train9\\weights\\best.pt')

# Create output directory to save frames with Fall Detected
output_folder = 'C:\\Users\\HP\\Desktop\\fall\\output_tree'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def process_frame(frame, frame_count, confidence_threshold=0.001):
    results = model(frame)

    for result in results:
        for box in result.boxes:
            # Get bounding box coordinates, class label, and confidence score
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_id = int(box.cls[0])  # Get the class ID
            confidence = box.conf[0]  # Get the confidence score

            # Only process if confidence exceeds the threshold
            if confidence >= confidence_threshold:
                # Define the color based on the class ID and detection confidence
                if class_id == 1:  # Class ID for fall detection
                    color = (0, 0, 255)  # Red for fall detection
                    label = f"Fall Detected ({confidence:.2f})"
                else:
                    color = (0, 255, 0)  # Green for other classes
                    label = f"Class {class_id} ({confidence:.2f})"

                # Draw the bounding box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return frame

# Function to process image or video
def process_input(input_source, confidence_threshold=0.001):
    frame_count = 0  # Track the frame number
    output_file_path = os.path.join(output_folder, os.path.basename(input_source))

    # Check if the input is a video file
    if input_source.endswith(('.mp4', '.avi', '.mov', '.gif', '.webm')):
        cap = cv2.VideoCapture(input_source)
        # Get the video details (frame width, height, FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_video = cv2.VideoWriter(output_file_path, fourcc, fps, (width, height))

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Process and display the frame
            processed_frame = process_frame(frame, frame_count, confidence_threshold)
            output_video.write(processed_frame)  # Write frame to output video file
            cv2.imshow('Detections', processed_frame)

            # Slow down the video display (e.g., 50 ms per frame)
            if cv2.waitKey(50) & 0xFF == ord('q'):  # 50ms = 20 frames per second (fps)
                break

            frame_count += 1  # Increment frame count

        cap.release()
        output_video.release()  # Release the VideoWriter object

    else:  # Assuming it's an image if not a video
        frame = cv2.imread(input_source)
        processed_frame = process_frame(frame, frame_count, confidence_threshold)

        # Save the processed image in the output folder
        output_image_path = os.path.join(output_folder, os.path.basename(input_source))
        cv2.imwrite(output_image_path, processed_frame)

        cv2.imshow('Detections', processed_frame)
        cv2.waitKey(0)

    cv2.destroyAllWindows()

# Example usage
input_path = 'C:\\Users\\HP\\Desktop\\fall\\T\\videoplayback (1).webm'  # Can be an image file or a video file
process_input(input_path, confidence_threshold=0.001)  # Confidence threshold can be adjusted



0: 640x384 (no detections), 41.8ms
Speed: 0.5ms preprocess, 41.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 33.4ms
Speed: 0.0ms preprocess, 33.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 31.6ms
Speed: 1.3ms preprocess, 31.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 30.1ms
Speed: 6.0ms preprocess, 30.1ms inference, 3.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 28.4ms
Speed: 8.8ms preprocess, 28.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 26.6ms
Speed: 2.2ms preprocess, 26.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 29.9ms
Speed: 0.0ms preprocess, 29.9ms inference, 5.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 32.4ms
Speed: 0.0ms preprocess, 32.4ms i

# FINAL COMBINED LOGIC

In [5]:
import cv2
import os
import math
from ultralytics import YOLO
import cvzone
import winsound  # Import winsound for sound alerts
import time  # Import time for controlling the beep duration

# Load the YOLOv8 model
model = YOLO('C:\\Users\\HP\\Desktop\\fall\\runs\\detect\\train9\\weights\\best.pt')

# Load class names from file
classnames = []
with open('C:\\Users\\HP\\Desktop\\fall\\classnames.txt', 'r') as f:
    classnames = f.read().splitlines()

# Create output directory to save frames with Fall Detected
output_folder = 'C:\\Users\\HP\\Desktop\\fall\\output_tree'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

temp = None  # For angle calculation
fall_confidences = []  # List to store confidence values for fall detection
frame_count_since_last_check = 0  # Frame counter for every 5 seconds

# Function to calculate the Euclidean distance between two points
def calculate_distance(p1, p2):
    return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

# Function to calculate the angle in degrees opposite to side c
def calculate_angle(a, b, c):
    return math.degrees(math.acos((a**2 + b**2 - c**2) / (2 * a * b)))

def process_frame(frame, frame_count, confidence_threshold=0.25):
    results = model(frame)

    for result in results:
        for box in result.boxes:
            # Get bounding box coordinates, class label, and confidence score
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]  # Get the confidence score
            class_detect = int(box.cls[0])
            class_detect = classnames[class_detect]
            conf = math.ceil(confidence * 100)

            if conf > confidence_threshold :
                # Calculate dimensions and centroid
                centroid = ((x1 + x2) / 2, (y1 + y2) / 2)
                upper_left = (x1, y1)
                upper_right = (x2, y1)
                height = y2 - y1
                width = x2 - x1

                # Human fall detection
                threshold = height - width
                a = calculate_distance(upper_left, upper_right)  # Base
                b = calculate_distance(upper_right, centroid)     # Side 2
                c = calculate_distance(upper_left, centroid)      # Side 3

                # Calculate angles
                angle_C = 180 - (calculate_angle(a, b, c) * 2)  # Angle at centroid

                if 0 < angle_C:
                    # Check fall condition based on angle
                    if (angle_C > 90 or angle_C < 10) or threshold < 0:
                        color = (0, 0, 255)  # Red for fall detection
                        label = f"Fall Detected ({confidence:.2f})"
                        fall_confidences.append(confidence)  # Store the confidence value

                        # Draw the bounding box and label
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    return frame

def continuous_beep(duration):
    end_time = time.time() + duration
    while time.time() < end_time:
        winsound.Beep(1500, 200)  # Frequency: 1500 Hz, Duration: 200 ms
        time.sleep(0.2)  # Short pause between beeps

# Function to process image or video
def process_input(input_source, confidence_threshold=0.25):
    global frame_count_since_last_check
    frame_count = 0  # Track the frame number
    output_file_path = os.path.join(output_folder, os.path.basename(input_source))

    # Check if the input is a video file
    if input_source.endswith(('.mp4', '.avi', '.mov', '.gif', '.webm')):
        cap = cv2.VideoCapture(input_source)
        # Get the video details (frame width, height, FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_video = cv2.VideoWriter(output_file_path, fourcc, fps, (width, height))

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Process and display the frame
            processed_frame = process_frame(frame, frame_count, confidence_threshold)
            output_video.write(processed_frame)  # Write frame to output video file
            cv2.imshow('Detections', processed_frame)

            # Increment the frame count
            frame_count += 1
            frame_count_since_last_check += 1
            
            # Every 5 seconds (fps * 5 seconds)
            if frame_count_since_last_check >= fps * 5:
                # Calculate average confidence
                if fall_confidences:
                    average_confidence = sum(fall_confidences) / len(fall_confidences) + 10
                    print(f"Average confidence over last 5 seconds: {average_confidence:.2f}")

                    # Check if the average confidence exceeds the threshold
                    if average_confidence > 0.30:
                        # Play a continuous warning beep sound for 3 seconds
                        continuous_beep(3)  # Beep for 3 seconds

                # Reset for the next 5 seconds
                fall_confidences.clear()
                frame_count_since_last_check = 0

            # Slow down the video display (e.g., 50 ms per frame)
            if cv2.waitKey(50) & 0xFF == ord('q'):  # 50ms = 20 frames per second (fps)
                break

        cap.release()
        output_video.release()  # Release the VideoWriter object

    else:  # Assuming it's an image if not a video
        frame = cv2.imread(input_source)
        processed_frame = process_frame(frame, frame_count, confidence_threshold)

        # Save the processed image in the output folder
        output_image_path = os.path.join(output_folder, os.path.basename(input_source))
        cv2.imwrite(output_image_path, processed_frame)

        cv2.imshow('Detections', processed_frame)
        cv2.waitKey(0)

    cv2.destroyAllWindows()

# Example usage
input_path = 'C:\\Users\\HP\\Desktop\\fall\\T\\IMP.mp4'  # Can be an image file or a video file
process_input(input_path, confidence_threshold=0.25)  # Confidence threshold can be adjusted



0: 480x640 2 0s, 48.6ms
Speed: 0.0ms preprocess, 48.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 0s, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 0s, 4.7ms
Speed: 2.2ms preprocess, 4.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 0s, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 0s, 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 0s, 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 0s, 9.5ms
Speed: 0.0ms preprocess, 9.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 0s, 4.1ms
Speed: 3.8ms preprocess, 4.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 0, 0.0ms
Speed: 0.0m